Make all imports and define the main graphical interface

In [1]:
%matplotlib widget

from imports import * 
from Head_content import head_content
from Left_content import left_content
from Center_content import  center_content
from Right_content import right_content
from plot_geometry import plot_geometry
from plot_incident_rays import plot_incident_rays
from plot_reflected_rays import plot_reflected_rays
from plot_zoom import plot_zoom
from plot_projection import plot_projection

plt.ioff()

H = head_content()
H_design = H.design()

C = center_content()
C_design = C.design()

R = right_content()
R_design = R.design()

L = left_content(center_instance=C, header_instance=H)
L_parabola = L.design_parabola()
L_ringArray = L.design_ring_array()
L_cylindrical = L.design_cylindrical()

L._left_content__variables_parabola()

app = widgets.AppLayout(header=H_design,
                        left_sidebar=L_parabola,
                        center=C_design,
                        right_sidebar=R_design,
                        footer=widgets.Button())
def changeLayout(*arg):
    global app
    ipv.clear()
    value = H.collector_type.value
    if value == 1:
        L._left_content__variables_parabola()
        app.left_sidebar = L_parabola
    if value == 2:
        L._left_content__variables_ringarray()
        app.left_sidebar = L_ringArray
    if value == 3:
        app.left_sidebar = L_cylindrical
        
def updateSourceGeometry(*arg):
    global app, L_parabola, L_ringArray, L_cylindrical
    L_parabola = L.design_parabola()
    L_ringArray = L.design_ring_array()
    L_cylindrical = L.design_cylindrical()
    
    value = H.collector_type.value
    if value == 1:
        app.left_sidebar = L_parabola
    if value == 2:
        app.left_sidebar = L_ringArray
    if value == 3:
        app.left_sidebar = L_cylindrical

H.collector_type.observe(changeLayout, 'value')
H.source_geometry.observe(updateSourceGeometry, 'value')


Define the process for plotting a collector surface

In [2]:
phi_tab = []
z_tab=[], 
rx=0.0
ry=0.0
inters=spy.Function
inters_lambda=any
yl_min=0.0
yl_max=0.0
zl_min=0.0
zl_max=0.0
parabola_rings=[]
h_2D = []
focs = []
# Update the list of varables for calculations
variables = L.variables
# Optical calculation classes instanciations
parabolic_calc = pmc((variables['parabola_focus_x'], variables['parabola_focus_y'], variables['parabola_height']), 
                                   variables['pt_source_coordinate'], variables['parabola_rot_angle'])
ring_calc = opra(Rin_0=variables['ringArray_internal_ray'], A_target=variables['ringArray_area'], 
                              N=variables['ringArray_N_rings'], w=variables['ringArray_material_width'], 
                              h_max=variables['ringArray_h_max'])
cyl_calc = cylc((variables['cylinder_length'], variables['cylinder_thikness'], variables['cylinder_height']), 
                (0, 1, 0 ), variables['pt_source_coordinate'], variables['parabola_rot_angle'])
# Geometry ploter instance
geo_plot = plot_geometry(fx=variables['parabola_focus_x'], fy=variables['parabola_focus_y'], h=variables['parabola_height'],
                         pt_source_pos=variables['pt_source_coordinate'], khoi=variables['parabola_rot_angle'], Rin_0=variables['ringArray_internal_ray'],
                         A_target=variables['ringArray_area'], N=variables['ringArray_N_rings'], w=variables['ringArray_material_width'], 
                         L=variables['cylinder_length'], th = variables['cylinder_thikness'], h_cyl = variables['cylinder_height'],
                         h_max=variables['ringArray_h_max'], n_theta=variables['n_theta'], n_phi=variables['n_phi'],
                         center_instance=C, left_instance=L, parabolic=parabolic_calc, ring=ring_calc, cylinder=cyl_calc)

def plot_collector_geometry(*arg):
    
    global phi_tab, z_tab, rx, ry, inters, inters_lambda, yl_min, yl_max, zl_min, zl_max, parabola_rings, h_2D, focs
    C.main_scene.scatters.clear()
    value = H.collector_type.value
    # Update the list of varables for calculations
    variables = L.variables
    # Optical calculation classes instanciations
    parabolic_calc.update_init((variables['parabola_focus_x'], variables['parabola_focus_y'], variables['parabola_height']),
                               variables['pt_source_coordinate'], variables['parabola_rot_angle'])
    ring_calc.update_init(Rin_0=variables['ringArray_internal_ray'], A_target=variables['ringArray_area'],
                          N=variables['ringArray_N_rings'], w=variables['ringArray_material_width'], 
                          h_max=variables['ringArray_h_max'])
    cyl_calc.update_init(surface=(variables['cylinder_length'], variables['cylinder_thikness'], variables['cylinder_height']),
                         sun_pos=variables['pt_source_coordinate'], khoi=variables['parabola_rot_angle'] )
    # Geometry ploter instance init update
    geo_plot.update_init(fx=variables['parabola_focus_x'], fy=variables['parabola_focus_y'], h=variables['parabola_height'],
                         L=variables['cylinder_length'], th = variables['cylinder_thikness'], h_cyl = variables['cylinder_height'],
                         pt_source_pos=variables['pt_source_coordinate'], khoi=variables['parabola_rot_angle'], Rin_0=variables['ringArray_internal_ray'],
                         A_target=variables['ringArray_area'], N=variables['ringArray_N_rings'], w=variables['ringArray_material_width'], 
                         h_max=variables['ringArray_h_max'], n_theta=variables['n_theta'], n_phi=variables['n_phi'],
                         center_instance=C, left_instance=L, parabolic=parabolic_calc, ring=ring_calc, cylinder=cyl_calc)
    if value == 1:
        phi_tab, z_tab, rx, ry, inters, inters_lambda, yl_min, yl_max, zl_min, zl_max = \
            geo_plot.plot_parabola()
    if value == 2:
        parabola_rings, h_2D, focs = geo_plot.plot_ringArray()
    if value == 3:
        phi_tab, z_tab, rx, ry, inters, inters_lambda, yl_min, yl_max, zl_min, zl_max = \
            geo_plot.plot_cylinder()
    else:
        L.result_label.value = 'error in plot_collector_geometry function'

L.plot_surf_bt.on_click(plot_collector_geometry)
    

Define the process for plotting incident rays to the collector

In [3]:
# Update the list of varables for calculations
variables = L.variables

# Incident rays plotter instance
inc_ray_plot = plot_incident_rays(n_phi=variables['n_phi'], n_theta=variables['n_theta'], inc_rayClr_value=variables['inc_rayClr_value'], h_cyl=variables['cylinder_height'],
                                  N=variables['ringArray_N_rings'], h=variables['parabola_height'], pt_source_pos=variables['pt_source_coordinate'],
                                  h_2D=h_2D, focs=focs, inters=inters, rx=rx, yl_min=yl_min,yl_max=yl_max, parabola_rings=parabola_rings,
                                  center_instance=C, left_instance=L, parabolic=parabolic_calc, ring=ring_calc, cylinder=cyl_calc)

def plot_incident_rays(*arg):
    # Update the list of varables for calculations
    variables = L.variables
    value = H.collector_type.value
    
    # incident rays plotter init update
    inc_ray_plot.update_init(n_phi=variables['n_phi'], n_theta=variables['n_theta'], inc_rayClr_value=variables['inc_rayClr_value'], h_cyl=variables['cylinder_height'],
                             N=variables['ringArray_N_rings'], h=variables['parabola_height'], pt_source_pos=variables['pt_source_coordinate'],
                             h_2D=h_2D, focs=focs, inters=inters, rx=rx, yl_min=yl_min,yl_max=yl_max, parabola_rings=parabola_rings,
                             center_instance=C, left_instance=L, parabolic=parabolic_calc, ring=ring_calc, cylinder=cyl_calc)
    if value == 1:
        inc_ray_plot.plot_incident_parabola()
    if value == 2:
        inc_ray_plot.plot_incident_ringArray()
    if value == 3:
        inc_ray_plot.plot_incident_cylinder()

L.plot_inc_bt.on_click(plot_incident_rays)

Define the process for plotting the reflected rays from the collector surface

In [4]:
# Update the list of varables for calculations
variables = L.variables

# Reflected rays plotter instance
refl_ray_plot = plot_reflected_rays(n_phi=variables['n_phi'], n_theta=variables['n_theta'], refl_rayClr_value=variables['refl_rayClr_value'], 
                                  N=variables['ringArray_N_rings'], h=variables['parabola_height'], pt_source_pos=variables['pt_source_coordinate'],
                                  h_2D=h_2D, focs=focs, inters=inters, rx=rx, yl_min=yl_min,yl_max=yl_max, parabola_rings=parabola_rings,
                                  center_instance=C, left_instance=L, parabolic=parabolic_calc)



def plot_reflected_rays(*arg):
    # Update the list of varables for calculations
    variables = L.variables
    value = H.collector_type.value
    
    # Reflected rays plotter init update
    refl_ray_plot.update_init(n_phi=variables['n_phi'], n_theta=variables['n_theta'], refl_rayClr_value=variables['refl_rayClr_value'], 
                              N=variables['ringArray_N_rings'], h=variables['parabola_height'], pt_source_pos=variables['pt_source_coordinate'],
                              h_2D=h_2D, focs=focs, inters=inters, rx=rx, yl_min=yl_min,yl_max=yl_max, parabola_rings=parabola_rings,
                              center_instance=C, left_instance=L, parabolic=parabolic_calc)
    
    if value == 1:
        refl_ray_plot.plot_reflected_parabola()
    if value == 2:
        refl_ray_plot.plot_reflected_ringArray()
    
L.plot_refl_bt.on_click(plot_reflected_rays)

Define the process for zomming on the reflected rays

In [5]:
# Update the list of varables for calculations
variables = L.variables

# Zoom plotting instance
zoom_plot = plot_zoom(n_phi=variables['n_phi'], n_theta=variables['n_theta'], h_2D=h_2D, focs=focs, 
                      N=variables['ringArray_N_rings'], pt_source_pos=variables['pt_source_coordinate'],
                      parabola_rings=parabola_rings, center_instance=C, right_instance=R, parabolic=parabolic_calc)

def plot_zoom_rays(*arg):
    # Update the list of varables for calculations
    variables = L.variables
    value = H.collector_type.value
    
    # Update init
    zoom_plot.update_init(n_phi=variables['n_phi'], n_theta=variables['n_theta'], h_2D=h_2D, focs=focs, 
                          N=variables['ringArray_N_rings'], pt_source_pos=variables['pt_source_coordinate'],
                          parabola_rings=parabola_rings, center_instance=C, right_instance=R, parabolic=parabolic_calc)
    
    if value==1:
        zoom_plot.plot_zoom_parabola()
    if value==2:
        zoom_plot.plot_zoom_ringArray()
        
C.plot_zoom_bt.on_click(plot_zoom_rays)
    

Define the process for making the 2D projction of the reflected rays

In [6]:
# Update the list of varables for calculations
variables = L.variables

# projection plotting instance
proj_plot = plot_projection(n_phi=variables['n_phi'], n_theta=variables['n_theta'], h_2D=h_2D, focs=focs, inters=inters,
                         N=variables['ringArray_N_rings'], pt_source_pos=variables['pt_source_coordinate'],
                         parabola_rings=parabola_rings, center_instance=C, right_instance=R, parabolic=parabolic_calc)

def plot_proj(*arg):
    # Update the list of varables for calculations
    variables = L.variables
    value = H.collector_type.value
    
    # Update init
    proj_plot.update_init(n_phi=variables['n_phi'], n_theta=variables['n_theta'], h_2D=h_2D, focs=focs, inters=inters,
                          N=variables['ringArray_N_rings'], pt_source_pos=variables['pt_source_coordinate'],
                          parabola_rings=parabola_rings, center_instance=C, right_instance=R, parabolic=parabolic_calc)
    
    if value==1:
        proj_plot.plot_proj_parabola()
    if value==2:
        proj_plot.plot_proj_ringArray()
        
C.plot_xy_bt.on_click(plot_proj)
    

Main graphical output

In [7]:
display(app)

AppLayout(children=(VBox(children=(HTML(value=" <h1 style='color:blue;margin-:100px'; text-align:center><u><b>…

Eq(1.0*sqrt(z**2), 0)


Function


ValueError: subs accepts either 1 or 2 arguments